In [32]:

import pandas as pd
import csv
import numpy as np
%matplotlib inline

# Loading the data/train/train.csv
df = pd.read_csv('/content/drive/My Drive/pet Adoption prediction dataset/train.csv')
test_data = pd.read_csv('/content/drive/My Drive/pet Adoption prediction dataset/test.csv')

In [33]:
df.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [34]:
df.isnull().sum()

pet_id               0
issue_date           0
listing_date         0
condition         1477
color_type           0
length(m)            0
height(cm)           0
X1                   0
X2                   0
breed_category       0
pet_category         0
dtype: int64

In [35]:
#dropping irrelevant columns
train_data = df.drop(['height(cm)'],axis=1)
train_data = train_data.drop(['length(m)'],axis=1)
test_data = test_data.drop(['height(cm)'],axis=1)
test_data = test_data.drop(['length(m)'],axis=1)
foo = test_data
train_data.head(5)

,pet_id,issue_date,listing_date,condition,color_type,X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,18,4,0.0,1


In [23]:
f_data = train_data.drop(['breed_category','pet_category'],axis=1)
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

f_data["condition"] = imputer.fit_transform(f_data[["condition"]]).ravel()
test_data["condition"] = imputer.fit_transform(test_data[["condition"]]).ravel()
test_data.head(5)

,pet_id,issue_date,listing_date,condition,color_type,X1,X2
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0,7
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0,7
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,7,1
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0,7


In [25]:
#Encoding Labels 
from sklearn.preprocessing import LabelEncoder

new_train_data = f_data.apply(LabelEncoder().fit_transform)
new_test_data1 = test_data.apply(LabelEncoder().fit_transform)
new_train_data.head(5)
new_test_data1.head()

,pet_id,issue_date,listing_date,condition,color_type,X1,X2
0,7521,131,3139,0,2,0,7
1,8012,2754,7669,2,36,0,1
2,2507,917,4650,2,2,0,7
3,5126,1467,4676,2,2,6,1
4,6870,2122,4821,2,14,0,7


In [26]:
#Standarization of data
from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()
scalar.fit(new_train_data)
new_train_data = scalar.transform(new_train_data)
new_train_data = pd.DataFrame(new_train_data)

scalar.fit(new_test_data1)
new_test_data1 = scalar.transform(new_test_data1)
new_test_data1 = pd.DataFrame(new_test_data1)


In [27]:
new_train_data.head()

,0,1,2,3,4,5,6
0,0.834022,0.273827,-1.496946,1.298727,-0.337623,1.161013,1.257279
1,0.454210,-0.777013,1.240735,0.443953,1.463576,1.161013,1.257279
2,0.815629,-0.451321,-1.378713,-0.410822,-0.492012,1.465326,-0.164116
3,1.058046,0.470150,1.345271,0.443953,1.463576,-0.817018,-1.016954
4,-0.706008,0.774281,-0.061436,1.298727,-1.161029,1.921795,-0.164116


In [28]:
x_train = new_train_data
x_test = new_test_data1
yb_train = train_data['pet_category']
ya_train = train_data['breed_category']


x_train.head(5)
yb_train.head()
ya_train.head()

0    0.0
1    0.0
2    2.0
3    0.0
4    0.0
Name: breed_category, dtype: float64

In [29]:
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier(n_estimators=175,max_features='log2',bootstrap=True,max_depth=17,min_samples_leaf=2,min_samples_split=3,n_jobs=-1,random_state=3)
clf.fit(x_train,ya_train)
y_predict1 = clf.predict(x_test)
y_predict1 = pd.DataFrame(y_predict1)   #prediction for the breed_category


clf.fit(x_train,yb_train)
y_predict2 = clf.predict(x_test)
y_predict2 = pd.DataFrame(y_predict2)
y_predict2.head()      

,0
0,2
1,1
2,2
3,2
4,2


In [36]:
final_df = foo.drop(['issue_date','listing_date','condition','color_type','X1','X2'],axis=1)
final_df['breed_category'] = y_predict1
final_df['pet_category'] = y_predict2
final_df.to_csv("/content/drive/My Drive/pet Adoption prediction dataset/output.csv", sep=',',index=True)
final_df.head

<bound method NDFrame.head of           pet_id  breed_category  pet_category
0     ANSL_75005             1.0             2
1     ANSL_76663             0.0             1
2     ANSL_58259             0.0             2
3     ANSL_67171             0.0             2
4     ANSL_72871             0.0             2
...          ...             ...           ...
8067  ANSL_66809             1.0             2
8068  ANSL_59041             1.0             2
8069  ANSL_60034             1.0             2
8070  ANSL_58066             2.0             4
8071  ANSL_69436             1.0             2

[8072 rows x 3 columns]>